In [1]:
# Fast small-MS write (metadata-only projection; no DATA rotation)
# - Keeps only the first N_TIMES_TO_KEEP timestamps to speed up IO
# - Recomputes UVW for a single ICRS/J2000 phase center
# - Writes to /tmp; change DEST_MS if you prefer a different location

import os, sys, time, shutil
import numpy as np
import astropy.units as u
from astropy.time import Time
from pyuvdata import UVData

# ----- Config -----
SRC_ROOT = "/data/dsa110-contimg/src"
IN_DIR = "/data/incoming_test/2025-09-05T03-12-56_HDF5"
IN_FILE = os.path.join(IN_DIR, "2025-09-05T03:12:56_sb00.hdf5")  # single subband for speed
DEST_MS = "/tmp/quick_small.ms"  # change to your preferred path
N_TIMES_TO_KEEP = 2              # keep first 2 times for a quick run
WRITE_MODEL = False              # set True to write MODEL_DATA with a point source

# ----- Path bootstrap for imports in notebook -----
if SRC_ROOT not in sys.path:
    sys.path.insert(0, SRC_ROOT)

from dsa110_contimg.conversion.helpers import (
    set_antenna_positions,
    _ensure_antenna_diameters,
    get_meridian_coords,
    set_model_column,
)
from dsa110_contimg.utils.fringestopping import calc_uvw_blt

# ----- Load UVH5 (single subband for speed) -----
t0 = time.time()
uv = UVData()
uv.read(
    IN_FILE,
    file_type="uvh5",
    run_check=False,
    run_check_acceptability=False,
    strict_uvw_antpos_check=False,
    check_extra=False,
)
uv.uvw_array = uv.uvw_array.astype(np.float64)

# ----- Optional: select first N_TIMES_TO_KEEP times to minimize size -----
unique_times = np.unique(uv.time_array)
if unique_times.size > N_TIMES_TO_KEEP:
    uv.select(times=unique_times[:N_TIMES_TO_KEEP], run_check=False)

# ----- Antenna geometry & diameters -----
set_antenna_positions(uv)
_ensure_antenna_diameters(uv)

# ----- Single-center (ICRS/J2000) phase center; recompute per-row UVW -----
pt_dec = uv.extra_keywords.get("phase_center_dec", 0.0) * u.rad
t_mid = Time(float(np.mean(uv.time_array)), format="jd").mjd
ra_icrs, dec_icrs = get_meridian_coords(pt_dec, t_mid)

# Build a single catalog entry
uv.phase_center_catalog = {}
pc_id = uv._add_phase_center(
    cat_name="FIELD_CENTER",
    cat_type="sidereal",
    cat_lon=float(ra_icrs.to_value(u.rad)),
    cat_lat=float(dec_icrs.to_value(u.rad)),
    cat_frame="icrs",
    cat_epoch=2000.0,
)
# Per-row ID array pointing to this center
if not hasattr(uv, "phase_center_id_array") or uv.phase_center_id_array is None:
    uv.phase_center_id_array = np.zeros(uv.Nblts, dtype=int)
uv.phase_center_id_array[:] = pc_id

# Compute UVW per time block (no DATA rotation)
nbls = uv.Nbls
times = np.unique(uv.time_array)
ant_pos = np.asarray(uv.antenna_positions)  # (Nants, 3) relative to telescope loc
ant1 = np.asarray(uv.ant_1_array[:nbls], dtype=int)
ant2 = np.asarray(uv.ant_2_array[:nbls], dtype=int)
blen = ant_pos[ant2, :] - ant_pos[ant1, :]  # (Nbls, 3)

for i, tval in enumerate(times):
    row_slice = slice(i * nbls, (i + 1) * nbls)
    time_vec = np.full(nbls, float(Time(tval, format="jd").mjd), dtype=float)
    uv.uvw_array[row_slice, :] = calc_uvw_blt(
        blen,
        time_vec,
        "J2000",
        ra_icrs,
        dec_icrs,
        obs="OVRO_MMA",
    )

# Mark as projected and ensure MS-friendly metadata
uv.phase_type = "phased"
uv.phase_center_frame = "icrs"
uv.phase_center_epoch = 2000.0
# Ascending frequency and positive channel width
uv.reorder_freqs(channel_order="freq", run_check=False)

print(f"Ntimes={uv.Ntimes}, Nbls={uv.Nbls}, Nblts={uv.Nblts}")
print("UVW finite fraction:", np.isfinite(uv.uvw_array).mean())

# ----- Write MS (fast path) -----
if os.path.exists(DEST_MS):
    shutil.rmtree(DEST_MS)

uv.write_ms(
    DEST_MS,
    clobber=True,
    run_check=False,
    check_extra=False,
    run_check_acceptability=False,
    strict_uvw_antpos_check=False,
    check_autos=False,
    fix_autos=False,
)
print(f"Wrote MS: {DEST_MS} in {time.time() - t0:.1f}s")

# ----- Optional: write MODEL_DATA for a point source at phase center -----
if WRITE_MODEL:
    set_model_column(DEST_MS[:-3], uv, pt_dec, ra_icrs, dec_icrs, flux_Jy=10.0)
    print("MODEL_DATA written.")


2025-10-07 17:49:03 [INFO] Setting DSA-110 antenna positions
2025-10-07 17:49:04 [INFO] Loaded dynamic antenna positions for 117 antennas


Ntimes=2, Nbls=4656, Nblts=9312
UVW finite fraction: 1.0
Wrote MS: /tmp/quick_small.ms in 12.9s


In [3]:
# Quick MS sanity checks for /tmp/quick_small.ms

import os, sys
import numpy as np

# CASA + casacore
from casatasks import listobs
from casacore.tables import table as tb

MS_PATH = "/tmp/quick_small.ms"

# 1) listobs to a file (non-interactive)
LISTOBS_PATH = "/tmp/quick_small.listobs"
listobs(vis=MS_PATH, listfile=LISTOBS_PATH, overwrite=True, verbose=True)
print("listobs written to:", LISTOBS_PATH)

# 2) FIELD names
with tb(MS_PATH + "::FIELD") as t:
    field_names = list(t.getcol("NAME"))
    phase_dir = t.getcol("PHASE_DIR")  # shape: (nrow, nfield?, 2) or (1, 1, 2)
    print("FIELDS:", field_names[:5], "(total:", len(field_names), ")")
    # Optional: print first PHASE_DIR (radians)
    if hasattr(phase_dir, "shape"):
        print("PHASE_DIR shape:", phase_dir.shape)

# 3) SPW summary
with tb(MS_PATH + "::SPECTRAL_WINDOW") as t:
    chan_freq = t.getcol("CHAN_FREQ")  # Hz, shape (nchan, nspw) or (nspw, nchan)
    chan_width = t.getcol("CHAN_WIDTH")  # Hz
    # Normalize shapes to (nchan,) for a single SPW
    cf = chan_freq.ravel(order="F") if chan_freq.ndim > 1 else chan_freq
    cw = chan_width.ravel(order="F") if chan_width.ndim > 1 else chan_width
    print("SPW: nchan =", cf.size, "freq ascending:", np.all(np.diff(cf) > 0))
    print("SPW: channel_width positive:", np.all(cw > 0))

# 4) UVW stats
with tb(MS_PATH) as t:
    uvw = t.getcol("UVW")  # shape (nrows, 3)
    print("UVW rows:", uvw.shape[0], "min(m):", np.nanmin(uvw), "max(m):", np.nanmax(uvw))
    # Read the shape of DATA without loading the whole column
    one = t.getcol("DATA", startrow=0, nrow=1)  # shape (npol, nchan, 1)
    print("DATA sample shape (npol, nchan, nrow=1):", one.shape)

# 5) Column presence (MODEL_DATA, CORRECTED_DATA)
with tb(MS_PATH) as t:
    cols = set(t.colnames())
    print("Has MODEL_DATA:", "MODEL_DATA" in cols, "Has CORRECTED_DATA:", "CORRECTED_DATA" in cols)


listobs written to: /tmp/quick_small.listobs
Successful readonly open of default-locked table /tmp/quick_small.ms::FIELD: 9 columns, 1 rows
FIELDS: ['FIELD_CENTER'] (total: 1 )
PHASE_DIR shape: (1, 1, 2)
Successful readonly open of default-locked table /tmp/quick_small.ms::SPECTRAL_WINDOW: 16 columns, 1 rows
SPW: nchan = 48 freq ascending: True
SPW: channel_width positive: True
Successful readonly open of default-locked table /tmp/quick_small.ms: 23 columns, 9312 rows
UVW rows: 9312 min(m): -1836.1816634101265 max(m): 1695.5881506802814
DATA sample shape (npol, nchan, nrow=1): (1, 48, 2)
Successful readonly open of default-locked table /tmp/quick_small.ms: 23 columns, 9312 rows
Has MODEL_DATA: False Has CORRECTED_DATA: False


In [ ]:
# Option A: metadata-only projection (no DATA rotation), write MS,
# optional MODEL_DATA if a calibrator is in-field

import os
import sys

# Ensure package path for imports when running in a notebook
SRC_ROOT = "/data/dsa110-contimg/src"
if SRC_ROOT not in sys.path:
    sys.path.insert(0, SRC_ROOT)

import shutil
import numpy as np
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord
from pyuvdata import UVData

from dsa110_contimg.conversion.helpers import (
    set_antenna_positions,
    _ensure_antenna_diameters,
    get_meridian_coords,
    set_model_column,
)

# ----------------------------
# Inputs/Outputs (quick test)
# ----------------------------
indir = "/data/incoming_test/2025-09-05T03-12-56_HDF5"
infile = os.path.join(indir, "2025-09-05T03:12:56_sb00.hdf5")
out_base = "/data/output/ms_test/quick_model_catalog"  # -> quick_model_catalog.ms
ms_path = out_base + ".ms"

# Clean any previous output
if os.path.exists(ms_path):
    shutil.rmtree(ms_path)

# ----------------------------
# Load a single UVH5 (fast) and prep metadata
# ----------------------------
uv = UVData()
uv.read(
    infile,
    file_type="uvh5",
    run_check=False,
    run_check_acceptability=False,
    strict_uvw_antpos_check=False,
    check_extra=False,
)
uv.uvw_array = uv.uvw_array.astype(np.float64)

# Antenna metadata (absolute positions + diameters)
set_antenna_positions(uv)
_ensure_antenna_diameters(uv)

# Pointing declination from metadata
pt_dec = uv.extra_keywords.get("phase_center_dec", 0.0) * u.rad

# ----------------------------
# Single-center fallback (ICRS/J2000) to avoid catalog edge cases
# ----------------------------
t_mjd = Time(float(uv.time_array.mean()), format="jd").mjd
ra_icrs, dec_icrs = get_meridian_coords(pt_dec, t_mjd)

# Reset catalog and assign one center
uv.phase_center_catalog = {}
pc_id = uv._add_phase_center(
    cat_name="FIELD_CENTER",
    cat_type="sidereal",
    cat_lon=float(ra_icrs.to_value(u.rad)),
    cat_lat=float(dec_icrs.to_value(u.rad)),
    cat_frame="icrs",
    cat_epoch=2000.0,
)

# Ensure per-row ID array and point every row to this center
if not hasattr(
        uv,
        "phase_center_id_array") or uv.phase_center_id_array is None:
    uv.phase_center_id_array = np.zeros(uv.Nblts, dtype=int)
uv.phase_center_id_array[:] = pc_id

# Mark as projected and coerce global frame/epoch
uv.phase_type = "phased"
uv.phase_center_frame = "icrs"
uv.phase_center_epoch = 2000.0

# Sanitize every entry in phase_center_catalog (defensive)
pc = getattr(uv, "phase_center_catalog", {}) or {}
id_arr = getattr(uv, "phase_center_id_array", None)
for _pc_id, entry in list(pc.items()):
    entry["cat_frame"] = "icrs"
    entry["cat_epoch"] = 2000.0
    entry.setdefault("cat_type", "sidereal")
    entry.setdefault("cat_name", f"phase{_pc_id}")
    if ("cat_lon" not in entry) or ("cat_lat" not in entry):
        if id_arr is not None and np.any(id_arr == _pc_id):
            t_mjd = Time(
                float(np.mean(uv.time_array[id_arr == _pc_id])), format="jd").mjd
        else:
            t_mjd = Time(float(np.mean(uv.time_array)), format="jd").mjd
        _ra_icrs, _dec_icrs = get_meridian_coords(pt_dec, t_mjd)
        entry["cat_lon"] = float(_ra_icrs.to_value(u.rad))
        entry["cat_lat"] = float(_dec_icrs.to_value(u.rad))
    entry.setdefault("cat_times", None)

# Sanity checks
print("Nblts, Ntimes, Nbls:", uv.Nblts, uv.Ntimes, uv.Nbls)
print("UVW finite fraction:", np.isfinite(uv.uvw_array).mean())
try:
    print("Unique frames in catalog:",
          {e.get("cat_frame") for e in uv.phase_center_catalog.values()})
except Exception:
    pass

# ----------------------------
# Write MS (no DATA rotation)
# ----------------------------
uv.write_ms(
    ms_path,
    clobber=True,
    run_check=False,
    check_extra=False,
    run_check_acceptability=False,
    strict_uvw_antpos_check=False,
    check_autos=False,
    fix_autos=False,
)
print("Wrote MS:", ms_path)

# ----------------------------
# Optional: MODEL_DATA only if a calibrator is within the PB half-power
# ----------------------------
CAT_PATH = "/data/dsa110-contimg/references/dsa110_hi/dsa110hi/resources/default_cal_catalog.csv"


def load_calibrator_catalog(path=CAT_PATH):
    dtype = [
        ("name", "U64"), ("ra", "U16"), ("dec", "U16"),
        ("equinox", "U16"), ("flux", "f8"), ("sidx", "f8"), ("nu0", "f8"),
    ]
    cat = np.genfromtxt(path, dtype=dtype, delimiter=",", comments="#")
    if cat.shape == ():  # normalize scalar row to array
        cat = np.array([cat], dtype=dtype)
    return cat


def compute_pb_fwhm_deg(freq_hz, dish_m=4.65):
    lam = 299_792_458.0 / float(freq_hz)
    return (1.2 * lam / dish_m * u.rad).to(u.deg).value


def find_calibrators_in_field(
        point_ra_rad,
        point_dec_rad,
        radius_deg,
        catalog):
    coords = SkyCoord(
        [f"{row['ra']} {row['dec']}" for row in catalog],
        unit=(u.hourangle, u.deg),
        frame="icrs",
        equinox="J2000",
    )
    field = SkyCoord(
        ra=point_ra_rad * u.rad,
        dec=point_dec_rad * u.rad,
        frame="icrs")
    sep = field.separation(coords).deg
    idx = np.where(sep <= radius_deg)[0]
    return [(int(i), float(sep[i])) for i in idx]


def calibrator_flux_at(freq_hz, row):
    s = float(row["flux"])
    sidx = row["sidx"]
    nu0 = row["nu0"]
    if np.isfinite(sidx) and np.isfinite(nu0) and nu0 > 0:
        return s * ((freq_hz / 1e9) / nu0) ** sidx
    return s


# Decide whether to write a model
freq_center_hz = float(np.mean(uv.freq_array))
pb_fwhm_deg = compute_pb_fwhm_deg(freq_center_hz, dish_m=4.65)
search_radius_deg = pb_fwhm_deg / 2.0

t_mjd = Time(float(uv.time_array.mean()), format="jd").mjd
point_ra, point_dec = get_meridian_coords(pt_dec, t_mjd)  # radians

catalog = load_calibrator_catalog(CAT_PATH)
matches = find_calibrators_in_field(
    point_ra.to_value(
        u.rad), point_dec.to_value(
            u.rad), search_radius_deg, catalog)

if not matches:
    print(
        f"No calibrators within {search_radius_deg:.2f} deg; skipping MODEL_DATA.")
else:
    matches.sort(key=lambda x: x[1])  # nearest
    idx, sep = matches[0]
    row = catalog[idx]
    cal = SkyCoord(
        f"{row['ra']} {row['dec']}",
        unit=(
            u.hourangle,
            u.deg),
        frame="icrs")
    flux_jy = calibrator_flux_at(freq_center_hz, row)
    print(
        f"Calibrator {row['name']} at sep {sep:.2f} deg; flux ~ {flux_jy:.2f} Jy @ {freq_center_hz/1e9:.2f} GHz")
    set_model_column(
        out_base, uv, pt_dec, cal.ra.to(
            u.rad), cal.dec.to(
            u.rad), flux_Jy=flux_jy)
    print("MODEL_DATA written to:", ms_path)


2025-10-07 17:28:52 [INFO] Setting DSA-110 antenna positions
2025-10-07 17:28:53 [INFO] Loaded dynamic antenna positions for 117 antennas


Nblts, Ntimes, Nbls: 111744 24 4656
UVW finite fraction: 1.0
Unique frames in catalog: {'icrs'}


In [3]:
# Quick iteration: 16 subbands → tiny MS in /dev/shm (RAM-backed)
# - Keeps first N times, first N channels, first N antennas
# - Metadata-only projection (no DATA rotation)
# - Per-time phase centers + UVW recompute
# - Ascending frequency
# - Estimates size vs /dev/shm free space

import os, sys, time, shutil, glob
import numpy as np
import astropy.units as u
from astropy.time import Time
from pyuvdata import UVData

# -----------------------------
# Config
# -----------------------------
SRC_ROOT = "/data/dsa110-contimg/src"
IN_DIR   = "/data/incoming_test/2025-09-05T03-12-56_HDF5"  # directory with 16 subbands
OUT_DIR  = "/dev/shm/ms_quick"                             # RAM-backed tmpfs for speed
START = "2025-09-05 03:12:00"
END   = "2025-09-05 03:13:30"

# Trim parameters (tune for speed)
N_TIMES_TO_KEEP = 1        # e.g., 1–2 times (fast)
N_CH_KEEP       = 96       # e.g., first 96 channels
N_ANTS_KEEP     = 32       # e.g., first 32 antennas

# Optional stability envs
os.environ.setdefault("HDF5_USE_FILE_LOCKING", "FALSE")
os.environ.setdefault("OMP_NUM_THREADS", "4")
os.environ.setdefault("MKL_NUM_THREADS", "4")

# Make package importable in this notebook
if SRC_ROOT not in sys.path:
    sys.path.insert(0, SRC_ROOT)

from dsa110_contimg.conversion import uvh5_to_ms_converter_v2 as v2
from dsa110_contimg.conversion.helpers import (
    set_antenna_positions,
    _ensure_antenna_diameters,
    get_meridian_coords,
)
from dsa110_contimg.utils.fringestopping import calc_uvw_blt

os.makedirs(OUT_DIR, exist_ok=True)

def write_tiny_ms_to_ram(in_dir, out_dir, start, end, n_times_keep=1, n_chan_keep=96, n_ants_keep=32):
    # Discover a complete subband group
    groups = v2.find_subband_groups(in_dir, start, end)
    if not groups:
        raise RuntimeError("No complete subband groups found in time window.")
    files = groups[0]
    base = os.path.splitext(os.path.basename(files[0]))[0].split("_sb")[0]
    ms_path = os.path.join(out_dir, f"{base}_tiny.ms")

    # Load & merge all 16 subbands
    uv = v2._load_and_merge_subbands(files)

    # 1) Keep first N times
    uniq_t = np.unique(uv.time_array)
    if uniq_t.size > n_times_keep:
        uv.select(times=uniq_t[:n_times_keep], run_check=False)

    # 2) Keep first N channels
    freqs = uv.freq_array.squeeze()
    if freqs.size > n_chan_keep:
        uv.select(freq_chans=np.arange(n_chan_keep), run_check=False)

    # 3) Keep first N antennas (based on those present)
    ant_ids_used = np.unique(np.r_[uv.ant_1_array[:uv.Nbls], uv.ant_2_array[:uv.Nbls]]).astype(int)
    if ant_ids_used.size > n_ants_keep:
        uv.select(antenna_nums=ant_ids_used[:n_ants_keep], run_check=False)

    # Antenna geometry & diameters
    set_antenna_positions(uv)
    _ensure_antenna_diameters(uv)

    # Per-time phase centers + UVW recompute (no DATA rotation)
    pt_dec = uv.extra_keywords.get("phase_center_dec", 0.0) * u.rad
    v2.set_per_time_phase_centers(uv, pt_dec)

    # Ascending frequency and mark as projected
    uv.reorder_freqs(channel_order="freq", run_check=False)
    uv.phase_type = "phased"

    # Coerce to ICRS/J2000 (defensive; set_per_time_phase_centers already does this)
    try:
        uv.phase_center_frame = "icrs"
        uv.phase_center_epoch = 2000.0
        if hasattr(uv, "phase_center_catalog") and uv.phase_center_catalog:
            for pc_id, entry in uv.phase_center_catalog.items():
                entry["cat_frame"] = "icrs"
                entry["cat_epoch"] = 2000.0
                entry.setdefault("cat_name", f"phase{pc_id}")
    except Exception:
        pass

    # Size estimate vs /dev/shm free space
    nrows = uv.Nblts
    nchan = uv.Nfreqs
    npols = uv.Npols
    bytes_data = nrows * nchan * npols * 8  # complex64 ~ 8B
    est_total = int(bytes_data * 3.0)       # rough overhead factor ×3 (DATA+FLAG+NSAMP+metadata)
    shm_total, shm_used, shm_free = shutil.disk_usage("/dev/shm")
    print(f"Estimate ~{est_total/1e9:.2f} GB; /dev/shm free ~{shm_free/1e9:.2f} GB")
    if est_total > shm_free * 0.9:
        print("Warning: Estimated MS may not fit into /dev/shm. Consider reducing N_TIMES/N_CH/N_ANTS.")
        # You can raise here if you prefer:
        # raise RuntimeError("Insufficient /dev/shm free space")

    # Write MS to RAM
    if os.path.exists(ms_path):
        shutil.rmtree(ms_path)
    t0 = time.time()
    uv.write_ms(
        ms_path,
        clobber=True,
        run_check=False,
        check_extra=False,
        run_check_acceptability=False,
        strict_uvw_antpos_check=False,
        check_autos=False,
        fix_autos=False,
    )
    dt = time.time() - t0

    return ms_path, uv, dt

ms_path, uv_tiny, write_secs = write_tiny_ms_to_ram(
    IN_DIR, OUT_DIR, START, END,
    n_times_keep=N_TIMES_TO_KEEP,
    n_chan_keep=N_CH_KEEP,
    n_ants_keep=N_ANTS_KEEP,
)

print(f"Wrote tiny MS: {ms_path} in {write_secs:.1f}s")
print("Ntimes =", uv_tiny.Ntimes, "Nbls =", uv_tiny.Nbls, "Nblts =", uv_tiny.Nblts)
print("UVW finite fraction:", np.isfinite(uv_tiny.uvw_array).mean())


KeyboardInterrupt: 

In [2]:
# Quick iteration → write a small MS to RAM-backed /dev/shm
# - Uses all 16 subbands
# - Keeps first N times for speed
# - Metadata-only projection (no DATA rotation), per-time centers, UVW recompute
# - Ascending frequency
# - Estimates size vs /dev/shm free space

from dsa110_contimg.utils.fringestopping import calc_uvw_blt
from dsa110_contimg.conversion.helpers import (
    set_antenna_positions,
    _ensure_antenna_diameters,
    get_meridian_coords,
)
from dsa110_contimg.conversion import uvh5_to_ms_converter_v2 as v2
import os
import sys
import time
import shutil
import glob
import numpy as np
import astropy.units as u
from astropy.time import Time
from pyuvdata import UVData

# -----------------------------
# Config
# -----------------------------
SRC_ROOT = "/data/dsa110-contimg/src"
IN_DIR = "/data/incoming_test/2025-09-05T03-12-56_HDF5"  # directory with 16 subbands
OUT_DIR = "/dev/shm/ms_quick"                             # RAM-backed tmpfs
# keep first N time stamps for fast write
N_TIMES_TO_KEEP = 2
START = "2025-09-05 03:12:00"
END = "2025-09-05 03:13:30"

# Optional stability envs
os.environ.setdefault("HDF5_USE_FILE_LOCKING", "FALSE")
os.environ.setdefault("OMP_NUM_THREADS", "4")
os.environ.setdefault("MKL_NUM_THREADS", "4")

# Make package importable in this notebook
if SRC_ROOT not in sys.path:
    sys.path.insert(0, SRC_ROOT)


os.makedirs(OUT_DIR, exist_ok=True)

# Reduce to a tiny working set for quick iteration
# 1) Keep 1 time
uniq_t = np.unique(uv_small.time_array) if 'uv_small' in globals() else np.unique(uv.time_array)
if uniq_t.size > 1:
    (uv_small if 'uv_small' in globals() else uv).select(times=uniq_t[:1], run_check=False)

# 2) Keep first 96 channels
freqs = (uv_small if 'uv_small' in globals() else uv).freq_array.squeeze()
n_ch_keep = min(96, freqs.size)
(uv_small if 'uv_small' in globals() else uv).select(freq_chans=np.arange(n_ch_keep), run_check=False)

# 3) Keep first 32 antennas
uv_obj = uv_small if 'uv_small' in globals() else uv
ant_ids_used = np.unique(np.r_[uv_obj.ant_1_array[:uv_obj.Nbls], uv_obj.ant_2_array[:uv_obj.Nbls]]).astype(int)
keep_ants = ant_ids_used[:min(32, ant_ids_used.size)]
uv_obj.select(antenna_nums=keep_ants, run_check=False)

# Recompute per-time centers + UVW (no DATA rotation) and mark as projected
from dsa110_contimg.conversion import uvh5_to_ms_converter_v2 as v2
pt_dec = uv_obj.extra_keywords.get("phase_center_dec", 0.0) * u.rad
v2.set_per_time_phase_centers(uv_obj, pt_dec)
uv_obj.reorder_freqs(channel_order="freq", run_check=False)
uv_obj.phase_type = "phased"
uv_obj.phase_center_frame = "icrs"; uv_obj.phase_center_epoch = 2000.0

print("Trimmed: Ntimes,Nbls,Nblts,Nchan,Npol:", uv_obj.Ntimes, uv_obj.Nbls, uv_obj.Nblts, uv_obj.Nfreqs, uv_obj.Npols)


NameError: name 'uv' is not defined